In [10]:
import pickle
import math

data_dir = "../../data/"

with open(data_dir + "train.txt", "r") as f:
     train_data = f.readlines()
Nodes = []
Edges = []
for i in range(len(train_data)):
    #if i%100 == 0:
        #print(i)
    nodes_list = [int(n) for n in train_data[i].split()]
    for node in nodes_list:
        Nodes.append(node)
    for node in nodes_list[1:]:
        Edges.append((nodes_list[0],node))
Nodes = set(Nodes)


In [132]:
# 找到所有的test_data中的点
data_dir = "../../data/"
with open(data_dir + "test-public.txt", "r") as f:
     test_data = f.readlines()
test_data = [i.split() for i in test_data[1:]]

nodes_in_test_data = set()
for edge in test_data:
    nodes_in_test_data.add(edge[1])
    nodes_in_test_data.add(edge[2])

In [133]:
len(nodes_in_test_data)

3948

In [51]:
data_dir = "../../data/"
def save_obj(obj, name ):
    with open( name + '.pkl', 'wb') as f:
        pickle.dump(obj, f, pickle.HIGHEST_PROTOCOL)

def load_obj(name):
    with open(data_dir + name + '.pkl', 'rb') as f:
        return pickle.load(f)
# load data
BasicFeatures = load_obj("pre_features-v2")

In [134]:
data_dir = "../../data/"

with open(data_dir + "edges.txt", "r") as f:
    read_rows = f.readlines()
    

rows = [i.split() for i in read_rows[0:]]
edges_of_all_test_nodes_related=list()

for row in rows:
    row[0] = int(row[0])
    row[1] = int(row[1])
    edge = (row[0], row[1])
    if str(row[0]) in nodes_in_test_data or str(row[1]) in nodes_in_test_data:
        edges_of_all_test_nodes_related.append(edge)

In [135]:
len(edges_of_all_test_nodes_related)

5068485

In [37]:
def save_obj(obj, name ):
    with open( name + '.pkl', 'wb') as f:
        pickle.dump(obj, f, pickle.HIGHEST_PROTOCOL)

def load_obj(name):
    with open(data_dir + name + '.pkl', 'rb') as f:
        return pickle.load(f)
# save_obj(edges_of_all_test_nodes_related,"edges_of_all_test_nodes_related")

In [6]:
import pickle

def load_obj(name):
    with open(name + '.pkl', 'rb') as f:
        return pickle.load(f)
data_dir = "../../data/"

import csv
filename='edges_of_all_test_nodes_related'
edges_of_all_test_nodes_related = load_obj(filename)
with open(filename + ".txt", 'w', encoding = 'utf8') as f:
    f_csv = csv.writer(f)
    f_csv.writerows(edges_of_all_test_nodes_related)


In [7]:
len(edges_of_all_test_nodes_related)

5068485

In [8]:
# 500w里选5w
import random
ps_edges = random.sample(edges_of_all_test_nodes_related, 50000)
len(ps_edges)
ps_edges[0]


(625451, 2593244)

In [9]:
nodes = set()
for edge in ps_edges:
    nodes.add(edge[0])
    nodes.add(edge[1])

In [16]:
len(ps_edges)
len(nodes)

47960

In [ ]:
from tqdm import tqdm

count = 0
final_edges=list()
while count < 50000:
    node1, node2 = random.sample(nodes,2)
    if (node1,node2) not in ps_edges:
        count+=1
        final_edges.append((node1,node2,0))

In [33]:
# final_edges
negative_edges = final_edges

In [35]:
for edge in ps_edges:
    final_edges.append((edge[0], edge[1], 1))

In [38]:
save_obj(final_edges,'final_edges')

In [42]:
len(final_edges)
# print(final_edges)

100000

In [43]:
# Genative labels
BasicFeatures = load_obj("pre_features-v2")
pre_features=BasicFeatures
#Cosine
def Cosine(Node1, Node2):
    n1 = pre_features[Node1]
    n2 = pre_features[Node2]
    common_neighors = list(set(n1[2]).intersection(n2[2]))
    lm = len(common_neighors)
    if lm == 0:
        return 0
    else:
        return (0.0+lm)/(len(n1[2])*len(n2[2]))

def get_jaccard_coefficient(source, sink):
    """
    in: source::Node object
    in: sink::Node object
    return: jaccard's cofficient::numeric
    """
    # transform
    neighbours_of_source_list = BasicFeatures[source][2]
    neighbours_of_sink_list = BasicFeatures[sink][2]
    
    neigbours_set_of_source = set(neighbours_of_source_list)
    neigbours_set_of_sink = set(neighbours_of_sink_list)
    union_neighbours = neigbours_set_of_source | neigbours_set_of_sink
    common_neighbours = neigbours_set_of_source & neigbours_set_of_sink
    if len(union_neighbours)==0:
        return 0.0
    return(len(common_neighbours)/len(union_neighbours))

def get_preferential_attachment(source, sink):
    # transform
    neighbours_of_source_list = BasicFeatures[source][2]
    neighbours_of_sink_list = BasicFeatures[sink][2]
    
    neigbours_set_of_source = set(neighbours_of_source_list)
    neigbours_set_of_sink = set(neighbours_of_sink_list)
    
    return len(neigbours_set_of_source)*len(neigbours_set_of_sink)

def get_adamic_adar(source, sink):
    # transform
    neighbours_of_source_list = BasicFeatures[source][2]
    neighbours_of_sink_list = BasicFeatures[sink][2]

    neigbours_set_of_source = set(neighbours_of_source_list)
    neigbours_set_of_sink = set(neighbours_of_sink_list)
    common_neighbours = neigbours_set_of_source & neigbours_set_of_sink
    # get the summation
    score = 0
    for common_node in common_neighbours:
        if math.log(len(BasicFeatures[common_node][2])) == 0:
            return 0.0
        score = score + 1/math.log(len(BasicFeatures[common_node][2]))
    return score

def get_resource_allocation(source, sink):
    neighbours_of_source_list = BasicFeatures[source][2]
    neighbours_of_sink_list = BasicFeatures[sink][2]
#     print(neighbours_of_source_list)
#     print(neighbours_of_sink_list)
    neigbours_set_of_source = set(neighbours_of_source_list)
    neigbours_set_of_sink = set(neighbours_of_sink_list)
    
    common_neighbours = neigbours_set_of_source & neigbours_set_of_sink
#     print(common_neighbours)
    score=0
    for common_node in common_neighbours:
        # number of the neighbours of the common_node
        try:
            single_common_node_score = 1/BasicFeatures[common_node][0]
        except:
            single_common_node_score=0
        score = score + single_common_node_score
    return score
    

# how similar are the outbound neighbors of source to sink
# either JA, PA, AA
def get_outbound_similarity_score(source, sink, metric):
    # get the outbound_node of source
    outbound_node_for_source_set = set(BasicFeatures[source][5])
    summation = 0
    for outbound_node_for_source in outbound_node_for_source_set:
        summation =summation + metric(sink,outbound_node_for_source)
    if len(outbound_node_for_source_set) == 0:
        return 0
    score = 1/len(outbound_node_for_source_set)*summation
    return score

# either JA, PA, AA
def get_inbound_similarity_score(source, sink, metric):
    # get the inbound_node of sink
    inbound_node_for_sink_set = set(BasicFeatures[source][3])
    summation = 0
    for inbound_node_for_sink in inbound_node_for_sink_set:
        summation =summation + metric(source,inbound_node_for_sink)
    if len(inbound_node_for_sink_set) == 0:
        return 0
    score = 1/len(inbound_node_for_sink_set)*summation
    return score

def get_common_neighbours(node1, node2):
    try:
        n1 = pre_features[node1]
        n2 = pre_features[node2]
        common_neighors = list(set(n1[2]).intersection(n2[2]))
        return common_neighors
    except:
        return 0


In [ ]:
filename='edges_10w'
with open(filename + ".txt", 'w', encoding = 'utf8') as f:
    f_csv = csv.writer(f)
    f_csv.writerows()


In [48]:
import pandas as pd
import math
training_df = pd.DataFrame()
for edge in final_edges:
    source = edge[0]
    sink = edge[1]
    label = edge[2]
    cosine = Cosine(source, sink)
    jaccard_coefficient = get_jaccard_coefficient(source, sink)
    preferential_attachment = get_preferential_attachment(source, sink)
    adamic_adar = get_adamic_adar(source, sink)
    resource_allocation = get_resource_allocation(source, sink)
    df_row = pd.DataFrame([source, sink, label, cosine, jaccard_coefficient, preferential_attachment, adamic_adar, resource_allocation]).T
    training_df = training_df.append(df_row)

In [ ]:
training_df.head

In [50]:
save_obj(training_df, 'training_df_10w')

In [53]:
def load_obj(name):
    with open( name + '.pkl', 'rb') as f:
        return pickle.load(f)

training_df_10w=load_obj( 'training_df_10w')

In [55]:
len(training_df_10w)

100000

In [54]:
training_df_10w.head

<bound method NDFrame.head of             0          1    2             3         4           5           6  \
0   4696690.0  1135200.0  0.0  9.264749e-05  0.009830    286656.0    2.981717   
0   4802205.0  4047144.0  0.0  0.000000e+00  0.000000       557.0    0.000000   
0   1951683.0  4024907.0  0.0  8.571429e-04  0.036585      7000.0    0.555249   
0   3542775.0  1239673.0  0.0  0.000000e+00  0.000000       464.0    0.000000   
0   2778303.0  1276807.0  0.0  0.000000e+00  0.000000       116.0    0.000000   
0   4842507.0  4546609.0  0.0  3.333333e-04  0.008499      8275.0    0.265828   
0   2050710.0  2353972.0  0.0  0.000000e+00  0.000000       155.0    0.000000   
0   3698046.0  3643090.0  0.0  1.221001e-03  0.004608       645.0    0.073802   
0   3672389.0  3959954.0  0.0  4.545455e-05  0.001859     19960.0    0.090343   
0   2384341.0  2356939.0  0.0  3.766478e-03  0.011236       531.0    0.170708   
0   3571391.0  1144219.0  0.0  0.000000e+00  0.000000        50.0    0.000000  

In [ ]:
# generate common neighbours
for edge in final_edges:
    source = edge[0]
    sink = edge[1]
    neighbours_of_source = BasicFeatures[source][]
    neighbours_of_sink = BasicFeatures[source][]
    

In [5]:
import csv
import time
'''
Description: get time
Input: 
Output: time
''' 
def nowtime():
    return time.strftime("%Y%m%d-%H%M", time.localtime())


"""
Description: Save prediction result to files
Input: (1) result
       (2) filename
Output: 
"""
def save_prediction_to_csv(result,filename):
#     headers = ['id','Prediction']
    with open(filename + str(nowtime()) + ".txt", 'w', encoding = 'utf8') as f:
        f_csv = csv.writer(f)
#         f_csv.writerow(headers)
        for row in result:
            f_csv.writerow(result)

save_prediction_to_csv(Edges,'edges')

TypeError: writerows() takes no keyword arguments